<a href="https://colab.research.google.com/github/HermiTManCode/HomeAssist_Voice/blob/main/Blender_render_script_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# coding: utf-8
"""
Interactive Blender Renderer for Google Colab.

This script creates a user-friendly interface to render Blender projects from
Google Drive. It features a smart, two-step file loading system, automatic
output path generation, and the ability to stop an ongoing render.

Version 3 Updates:
- Added a "Stop Render" button to terminate the rendering process.
- Implemented a two-step file loading system:
  1. Select a Project Folder.
  2. Select a Shot/.blend file from within that folder.
- The script intelligently finds blend files named `shot##.blend` or `foldername.blend`.
- Output paths are now automatically created to match the project/shot structure.
"""

# --- Core Imports ---
import os
import sys
import requests
import glob
import re
import subprocess
from bs4 import BeautifulSoup

# --- Environment-Specific Imports (for Colab) ---
try:
    from google.colab import drive
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    _IS_COLAB = True
except ImportError:
    print("Warning: This script is designed for Google Colab and requires 'ipywidgets'.")
    _IS_COLAB = False

# --- Main Application Class ---
class BlenderColabRenderer:
    """Manages the UI and rendering process for Blender in Colab."""

    def __init__(self):
        """Initializes the application, UI, and state variables."""
        if not _IS_COLAB:
            sys.exit("Execution stopped: Not running in a Colab environment.")

        self.render_process = None
        self._mount_drive()
        self._build_ui()
        self._bind_events()
        self.refresh_project_folders()

    def _mount_drive(self):
        """Mounts Google Drive if not already mounted."""
        if not os.path.exists('/content/drive'):
            print("🗂️ Mounting Google Drive...")
            drive.mount('/content/drive')
        else:
            print("✅ Google Drive is already mounted.")

    def _build_ui(self):
        """Creates and arranges all the ipywidgets for the UI."""
        style = {'description_width': 'initial'}
        layout = widgets.Layout(width='95%')

        self.projects_path_input = widgets.Text(value='/content/drive/MyDrive/blender_projects', description='Projects Path:', style=style, layout=layout)
        self.refresh_button = widgets.Button(description="🔄 Refresh Project List", button_style='info', icon='refresh')
        self.blender_version_dropdown = widgets.Dropdown(description='Blender Version:', style=style, layout=layout, disabled=True)
        self.project_folder_dropdown = widgets.Dropdown(description='Project Folder:', style=style, layout=layout, disabled=True)
        self.shot_file_dropdown = widgets.Dropdown(description='Shot / Blend File:', style=style, layout=layout, disabled=True)
        self.render_engine_dropdown = widgets.Dropdown(options=['CYCLES', 'EEVEE', 'WORKBENCH'], value='CYCLES', description='Render Engine:', style=style, layout=layout)
        self.device_dropdown = widgets.Dropdown(options=['CUDA', 'OPTIX', 'HIP', 'CPU'], value='CUDA', description='Device:', style=style, layout=layout)
        self.output_format_dropdown = widgets.Dropdown(options=[('Default (from .blend)', 'DEFAULT'), ('PNG', 'PNG'), ('OpenEXR (Multilayer)', 'OPEN_EXR_MULTILAYER')], value='DEFAULT', description='Output Format:', style=style, layout=layout)
        self.start_frame_input = widgets.IntText(value=1, description='Start:', style=style, layout=widgets.Layout(width='47%'))
        self.end_frame_input = widgets.IntText(value=10, description='End:', style=style, layout=widgets.Layout(width='47%'))
        self.run_button = widgets.Button(description="Start Render", button_style='success', icon='play', disabled=True)
        self.stop_button = widgets.Button(description="Stop Render", button_style='danger', icon='stop', disabled=True)
        self.output_log = widgets.Output()

        frame_box = widgets.HBox([self.start_frame_input, self.end_frame_input], layout=widgets.Layout(justify_content='space-between'))
        control_buttons = widgets.HBox([self.run_button, self.stop_button])

        self.ui = widgets.VBox([
            widgets.HTML("<h3>1. Select Project</h3>"), self.projects_path_input, self.refresh_button,
            self.project_folder_dropdown, self.shot_file_dropdown,
            widgets.HTML("<hr><h3>2. Set Render Options</h3>"),
            self.blender_version_dropdown, self.render_engine_dropdown, self.device_dropdown,
            self.output_format_dropdown, frame_box,
            widgets.HTML("<hr>"), control_buttons
        ])

    def _bind_events(self):
        """Binds widget events to their handler functions."""
        self.refresh_button.on_click(self.refresh_project_folders)
        self.project_folder_dropdown.observe(self.on_project_folder_change, names='value')
        self.run_button.on_click(self.run_render)
        self.stop_button.on_click(self.stop_render)

    def display_app(self):
        """Displays the assembled UI and the output log."""
        display(self.ui, self.output_log)

    # --- UI Update and Data Fetching Logic ---
    def refresh_project_folders(self, b=None):
        """Scans the projects path for subdirectories."""
        with self.output_log:
            clear_output(wait=True)
            print("⏳ Fetching Blender versions and scanning for project folders...")

            # Fetch Blender versions
            blender_versions = self._get_blender_versions()
            if blender_versions:
                self.blender_version_dropdown.options = blender_versions
                self.blender_version_dropdown.disabled = False
                print("✅ Blender versions loaded.")
            else:
                print("❌ Failed to load Blender versions.")

            # Find project folders
            path = self.projects_path_input.value
            if not os.path.isdir(path):
                print(f"⚠️ Projects path does not exist: {path}")
                self.project_folder_dropdown.options = []
                self.run_button.disabled = True
                return

            try:
                folders = sorted([d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))])
                if folders:
                    self.project_folder_dropdown.options = folders
                    self.project_folder_dropdown.disabled = False
                    print(f"✅ Found {len(folders)} project folder(s).")
                else:
                    self.project_folder_dropdown.options = []
                    self.project_folder_dropdown.disabled = True
                    print(f"❌ No project folders found in '{path}'.")
                # Trigger update for the first folder in the list
                self.on_project_folder_change({'new': self.project_folder_dropdown.value})
            except Exception as e:
                print(f"❌ Error scanning for folders: {e}")

    def on_project_folder_change(self, change):
        """Updates the shot file dropdown when a project folder is selected."""
        folder_path = os.path.join(self.projects_path_input.value, change['new'])
        if not os.path.isdir(folder_path):
            self.shot_file_dropdown.options = []
            self.shot_file_dropdown.disabled = True
            self.run_button.disabled = True
            return

        # Smart file discovery logic
        folder_name = os.path.basename(folder_path)
        main_blend_file = os.path.join(folder_path, f"{folder_name}.blend")
        shot_files = glob.glob(os.path.join(folder_path, "shot*.blend"))

        valid_files = []
        if os.path.exists(main_blend_file):
            valid_files.append(os.path.basename(main_blend_file))
        valid_files.extend([os.path.basename(f) for f in shot_files])

        if valid_files:
            self.shot_file_dropdown.options = sorted(valid_files)
            self.shot_file_dropdown.disabled = False
            self.run_button.disabled = False
        else:
            self.shot_file_dropdown.options = []
            self.shot_file_dropdown.disabled = True
            self.run_button.disabled = True
            with self.output_log:
                print(f"ℹ️ No valid blend files (`{folder_name}.blend` or `shot##.blend`) found in '{folder_name}'.")

    def _get_blender_versions(self):
        """Scrapes the Blender website for available Linux versions."""
        try:
            url = 'https://download.blender.org/release/'
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            version_pattern = re.compile(r'blender-(\d+\.\d+\.\d+)-linux-x64\.tar\.xz')
            versions = set()
            for a in soup.find_all('a', href=True):
                major_match = re.match(r'Blender(\d\.\d)/', a['href'])
                if major_match:
                    major_version_url = f"{url}{a['href']}"
                    sub_response = requests.get(major_version_url, timeout=10)
                    sub_soup = BeautifulSoup(sub_response.text, 'html.parser')
                    for sub_a in sub_soup.find_all('a', href=True):
                        match = version_pattern.search(sub_a['href'])
                        if match: versions.add(match.group(1))
            if not versions: raise ValueError("No versions found.")
            return sorted(list(versions), reverse=True)
        except Exception as e:
            with self.output_log: print(f"❌ Error fetching Blender versions: {e}"); return []

    # --- Render Control Logic ---
    def _set_render_state(self, is_rendering):
        """Toggles the state of UI elements during rendering."""
        self.run_button.disabled = is_rendering
        self.stop_button.disabled = not is_rendering
        # Disable other UI elements to prevent changes during render
        for widget in [self.projects_path_input, self.refresh_button, self.blender_version_dropdown,
                       self.project_folder_dropdown, self.shot_file_dropdown, self.render_engine_dropdown,
                       self.device_dropdown, self.output_format_dropdown, self.start_frame_input, self.end_frame_input]:
            widget.disabled = is_rendering

    def stop_render(self, b=None):
        """Stops the currently running Blender subprocess."""
        with self.output_log:
            if self.render_process and self.render_process.poll() is None:
                print("\n🛑 Sending termination signal to Blender...")
                self.render_process.terminate()
                try:
                    self.render_process.wait(timeout=10)
                    print("✅ Render process stopped by user.")
                except subprocess.TimeoutExpired:
                    print("⚠️ Blender did not terminate gracefully. Forcing kill.")
                    self.render_process.kill()
            else:
                print("ℹ️ No active render process to stop.")
        self._set_render_state(False)

    def run_render(self, b=None):
        """Handles the entire process of downloading, unpacking, and rendering."""
        with self.output_log:
            clear_output(wait=True)
            self._set_render_state(True)

            # 1. Download and Extract Blender
            blender_version = self.blender_version_dropdown.value
            blender_filename = f'blender-{blender_version}-linux-x64.tar.xz'
            drive_blender_path = f'/content/drive/MyDrive/blender_versions/{blender_filename}'
            os.makedirs(os.path.dirname(drive_blender_path), exist_ok=True)
            if not os.path.exists(f"/content/blender-{blender_version}-linux-x64"):
                if not os.path.exists(drive_blender_path):
                    print(f"📥 Downloading Blender {blender_version}...")
                    major_version = '.'.join(blender_version.split('.')[:2])
                    download_url = f'https://download.blender.org/release/Blender{major_version}/{blender_filename}'
                    try:
                        response = requests.get(download_url, stream=True, timeout=15)
                        response.raise_for_status()
                        with open(drive_blender_path, 'wb') as f:
                            for chunk in response.iter_content(chunk_size=8192): f.write(chunk)
                        print(f"✅ Download complete.")
                    except Exception as e:
                        print(f"❌ Download failed: {e}"); self._set_render_state(False); return
                else:
                    print(f"✅ Blender {blender_version} found in Google Drive.")

                print("📦 Unpacking Blender...")
                os.system(f"tar -xf '{drive_blender_path}' -C /content/")
                print("✅ Blender is ready.")
            else:
                 print("✅ Blender is already unpacked.")

            # 2. Construct Paths and Render Command
            project_folder = self.project_folder_dropdown.value
            shot_file = self.shot_file_dropdown.value
            blend_file_path = os.path.join(self.projects_path_input.value, project_folder, shot_file)

            # Smart output path
            shot_name = os.path.splitext(shot_file)[0]
            if shot_name == project_folder: # Case: project_A/project_A.blend
                output_dir = os.path.join('/content/drive/MyDrive/RENDER_OUTPUT', project_folder, '')
                output_prefix = f"{shot_name}_####"
            else: # Case: project_A/shot01.blend
                output_dir = os.path.join('/content/drive/MyDrive/RENDER_OUTPUT', project_folder, shot_name, '')
                output_prefix = f"{shot_name}_####"
            os.makedirs(output_dir, exist_ok=True)

            output_format = self.output_format_dropdown.value
            format_flag = f"-F {output_format}" if output_format != 'DEFAULT' else ""

            render_command = (
                f"/content/blender-{blender_version}-linux-x64/blender -b '{blend_file_path}' "
                f"-noaudio -E '{self.render_engine_dropdown.value}' "
                f"-o '{output_dir}{output_prefix}' {format_flag} "
                f"-s {self.start_frame_input.value} -e {self.end_frame_input.value} -a "
                f"-- --cycles-device {self.device_dropdown.value}"
            )

            print("\n" + "="*50)
            print(f"🎬 Starting render for: {shot_file}")
            print(f"💾 Output will be saved to: {output_dir}")
            print(f"🔧 Full Command: {render_command}")
            print("="*50 + "\n")

            # 3. Execute Render and Stream Output
            try:
                self.render_process = subprocess.Popen(
                    render_command, shell=True, stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT, text=True, bufsize=1
                )

                for line in iter(self.render_process.stdout.readline, ''):
                    print(line, end='')
                    if "Saved:" in line:
                        match = re.search(r'Fra:(\d+)', line)
                        if match: print(f"\n✨✨✨ Frame {match.group(1)} rendered successfully. ✨✨✨\n")

                self.render_process.stdout.close()
                return_code = self.render_process.wait()

                if return_code == 0:
                    print("\n" + "="*50, "\n🎉 Animation rendering complete!", "="*50 + "\n")
                elif self.render_process.returncode is not None : # Stopped by user, don't show error
                    pass
                else:
                    print(f"\n⚠️ Render process finished with errors (code: {return_code}). Check log.")

            except Exception as e:
                print(f"\n❌ An error occurred while running the render command: {e}")
            finally:
                self.render_process = None
                self._set_render_state(False)

# --- Script Entry Point ---
if __name__ == '__main__' and _IS_COLAB:
    app = BlenderColabRenderer()
    app.display_app()

🗂️ Mounting Google Drive...
